In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [95.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchp

In [2]:
spark

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls './drive/My Drive/Colab Notebooks/Spark/movielens'

links.csv  movies.csv  movies.gsheet  ratings.csv  tags.csv


Import all the `sql.functions` and `sql.types` from `pyspark`

In [5]:
from pyspark.sql.functions import *
from pyspark.sql import types

Specify the path for the data. In my case, it is mounted on google drive

In [6]:
fpath = '/content/drive/My Drive/Colab Notebooks/Spark/movielens'

Read the `csv` data file and make sure to set the `header` to `True` so that the schema is inferred automatically.

In [7]:
movdf = spark.read.csv(fpath+"/movies.csv", header=True)

In [8]:
movdf.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



Modify the data type of `movieId` column entries from string to `integer`.

In [9]:
movdf.take(10)

[Row(movieId='1', title='Toy Story (1995)', genres='Adventure|Animation|Children|Comedy|Fantasy'),
 Row(movieId='2', title='Jumanji (1995)', genres='Adventure|Children|Fantasy'),
 Row(movieId='3', title='Grumpier Old Men (1995)', genres='Comedy|Romance'),
 Row(movieId='4', title='Waiting to Exhale (1995)', genres='Comedy|Drama|Romance'),
 Row(movieId='5', title='Father of the Bride Part II (1995)', genres='Comedy'),
 Row(movieId='6', title='Heat (1995)', genres='Action|Crime|Thriller'),
 Row(movieId='7', title='Sabrina (1995)', genres='Comedy|Romance'),
 Row(movieId='8', title='Tom and Huck (1995)', genres='Adventure|Children'),
 Row(movieId='9', title='Sudden Death (1995)', genres='Action'),
 Row(movieId='10', title='GoldenEye (1995)', genres='Action|Adventure|Thriller')]

In [10]:
movdf = movdf.withColumn("movieId", col("movieId").cast(types.IntegerType()))

`movdf` dataframe now has `movieId` whose entries are now of `integer` type.

In [11]:
movdf.take(10)

[Row(movieId=1, title='Toy Story (1995)', genres='Adventure|Animation|Children|Comedy|Fantasy'),
 Row(movieId=2, title='Jumanji (1995)', genres='Adventure|Children|Fantasy'),
 Row(movieId=3, title='Grumpier Old Men (1995)', genres='Comedy|Romance'),
 Row(movieId=4, title='Waiting to Exhale (1995)', genres='Comedy|Drama|Romance'),
 Row(movieId=5, title='Father of the Bride Part II (1995)', genres='Comedy'),
 Row(movieId=6, title='Heat (1995)', genres='Action|Crime|Thriller'),
 Row(movieId=7, title='Sabrina (1995)', genres='Comedy|Romance'),
 Row(movieId=8, title='Tom and Huck (1995)', genres='Adventure|Children'),
 Row(movieId=9, title='Sudden Death (1995)', genres='Action'),
 Row(movieId=10, title='GoldenEye (1995)', genres='Action|Adventure|Thriller')]

Read the `ratings.csv` to `ratedf` while inferring the schema by setting `header` to `True`

In [12]:
ratedf = spark.read.csv(fpath+"/ratings.csv", header=True)

Cast `userId`, `movieId` and `rating` as `integer` type.

In [13]:
ratedf = ratedf.withColumn("userId", col("userId").cast(types.IntegerType())).withColumn("movieId", col("movieId").cast(types.IntegerType())).withColumn("rating", col("rating").cast(types.IntegerType()))

In [14]:
ratedf.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: string (nullable = true)



In [15]:
ratedf.take(10)

[Row(userId=1, movieId=1, rating=4, timestamp='964982703'),
 Row(userId=1, movieId=3, rating=4, timestamp='964981247'),
 Row(userId=1, movieId=6, rating=4, timestamp='964982224'),
 Row(userId=1, movieId=47, rating=5, timestamp='964983815'),
 Row(userId=1, movieId=50, rating=5, timestamp='964982931'),
 Row(userId=1, movieId=70, rating=3, timestamp='964982400'),
 Row(userId=1, movieId=101, rating=5, timestamp='964980868'),
 Row(userId=1, movieId=110, rating=4, timestamp='964982176'),
 Row(userId=1, movieId=151, rating=5, timestamp='964984041'),
 Row(userId=1, movieId=157, rating=5, timestamp='964984100')]

Rename the `movieId` column to `mId` preemptively as `movdf` also has `movieId` column and this would be ambiguous in the event of a join.

In [16]:
ratedf = ratedf.withColumnRenamed("movieId", "mId")

In [17]:
ratedf.take(10)

[Row(userId=1, mId=1, rating=4, timestamp='964982703'),
 Row(userId=1, mId=3, rating=4, timestamp='964981247'),
 Row(userId=1, mId=6, rating=4, timestamp='964982224'),
 Row(userId=1, mId=47, rating=5, timestamp='964983815'),
 Row(userId=1, mId=50, rating=5, timestamp='964982931'),
 Row(userId=1, mId=70, rating=3, timestamp='964982400'),
 Row(userId=1, mId=101, rating=5, timestamp='964980868'),
 Row(userId=1, mId=110, rating=4, timestamp='964982176'),
 Row(userId=1, mId=151, rating=5, timestamp='964984041'),
 Row(userId=1, mId=157, rating=5, timestamp='964984100')]

Join `movdef` with `ratedf` on the join predicate `movieId == mId` 

In [18]:
joindf = movdf.join(ratedf, movdf["movieId"] == ratedf["mId"])

In [19]:
joindf.take(10)

[Row(movieId=1, title='Toy Story (1995)', genres='Adventure|Animation|Children|Comedy|Fantasy', userId=1, mId=1, rating=4, timestamp='964982703'),
 Row(movieId=3, title='Grumpier Old Men (1995)', genres='Comedy|Romance', userId=1, mId=3, rating=4, timestamp='964981247'),
 Row(movieId=6, title='Heat (1995)', genres='Action|Crime|Thriller', userId=1, mId=6, rating=4, timestamp='964982224'),
 Row(movieId=47, title='Seven (a.k.a. Se7en) (1995)', genres='Mystery|Thriller', userId=1, mId=47, rating=5, timestamp='964983815'),
 Row(movieId=50, title='Usual Suspects, The (1995)', genres='Crime|Mystery|Thriller', userId=1, mId=50, rating=5, timestamp='964982931'),
 Row(movieId=70, title='From Dusk Till Dawn (1996)', genres='Action|Comedy|Horror|Thriller', userId=1, mId=70, rating=3, timestamp='964982400'),
 Row(movieId=101, title='Bottle Rocket (1996)', genres='Adventure|Comedy|Crime|Romance', userId=1, mId=101, rating=5, timestamp='964980868'),
 Row(movieId=110, title='Braveheart (1995)', genre

Drop the `mId` column from `joindf` as it's the same coumn as `movieId`.

In [20]:
joindf = joindf.drop("mId")

In [21]:
joindf.take(20)

[Row(movieId=1, title='Toy Story (1995)', genres='Adventure|Animation|Children|Comedy|Fantasy', userId=1, rating=4, timestamp='964982703'),
 Row(movieId=3, title='Grumpier Old Men (1995)', genres='Comedy|Romance', userId=1, rating=4, timestamp='964981247'),
 Row(movieId=6, title='Heat (1995)', genres='Action|Crime|Thriller', userId=1, rating=4, timestamp='964982224'),
 Row(movieId=47, title='Seven (a.k.a. Se7en) (1995)', genres='Mystery|Thriller', userId=1, rating=5, timestamp='964983815'),
 Row(movieId=50, title='Usual Suspects, The (1995)', genres='Crime|Mystery|Thriller', userId=1, rating=5, timestamp='964982931'),
 Row(movieId=70, title='From Dusk Till Dawn (1996)', genres='Action|Comedy|Horror|Thriller', userId=1, rating=3, timestamp='964982400'),
 Row(movieId=101, title='Bottle Rocket (1996)', genres='Adventure|Comedy|Crime|Romance', userId=1, rating=5, timestamp='964980868'),
 Row(movieId=110, title='Braveheart (1995)', genres='Action|Drama|War', userId=1, rating=4, timestamp='9

Number of rows in `joindf`

In [22]:
joindf.count()

100836

`Persist` the `joindf` in memory as it will be used for the rest of the operations

In [23]:
joindf.persist()

DataFrame[movieId: int, title: string, genres: string, userId: int, rating: int, timestamp: string]

Group the data by the `rating` column to get the `count` of each `rating` category.

In [24]:
joindf.groupby("rating").count().show()

+------+-----+
|rating|count|
+------+-----+
|     1| 4602|
|     3|33183|
|     5|13211|
|     4|35369|
|     2|13101|
|     0| 1370|
+------+-----+



#Movies with 5 star ratings 

In [25]:
joindf.filter(joindf.rating > 4).count()

13211

Get the titles of movies that were rated 5 stars. Limit the results to 10

In [26]:
joindf.filter(joindf.rating == 5).select(joindf.title).take(10)

[Row(title='Seven (a.k.a. Se7en) (1995)'),
 Row(title='Usual Suspects, The (1995)'),
 Row(title='Bottle Rocket (1996)'),
 Row(title='Rob Roy (1995)'),
 Row(title='Canadian Bacon (1995)'),
 Row(title='Desperado (1995)'),
 Row(title='Billy Madison (1995)'),
 Row(title='Dumb & Dumber (Dumb and Dumber) (1994)'),
 Row(title='Star Wars: Episode IV - A New Hope (1977)'),
 Row(title='Tommy Boy (1995)')]

Another way to check the total number of elements in the dataframe is to `groupby()` on `rating` column and count it.

In [27]:
tempdf = joindf.groupby("rating").count()

In [28]:
tempdf.printSchema()

root
 |-- rating: integer (nullable = true)
 |-- count: long (nullable = false)



Then, simply do a `sum` on `count` column.

In [29]:
tempdf.select(sum("count")).show()

+----------+
|sum(count)|
+----------+
|    100836|
+----------+



In [30]:
joindf.select("*").where(joindf.title == "Rob Roy (1995)").show()

+-------+--------------+--------------------+------+------+----------+
|movieId|         title|              genres|userId|rating| timestamp|
+-------+--------------+--------------------+------+------+----------+
|    151|Rob Roy (1995)|Action|Drama|Roma...|     1|     5| 964984041|
|    151|Rob Roy (1995)|Action|Drama|Roma...|     6|     4| 845553586|
|    151|Rob Roy (1995)|Action|Drama|Roma...|    33|     4| 939715904|
|    151|Rob Roy (1995)|Action|Drama|Roma...|    38|     5| 841341494|
|    151|Rob Roy (1995)|Action|Drama|Roma...|    40|     4| 832059189|
|    151|Rob Roy (1995)|Action|Drama|Roma...|    57|     3| 965799554|
|    151|Rob Roy (1995)|Action|Drama|Roma...|    64|     3|1161529309|
|    151|Rob Roy (1995)|Action|Drama|Roma...|    75|     2|1158967103|
|    151|Rob Roy (1995)|Action|Drama|Roma...|    91|     3|1112710950|
|    151|Rob Roy (1995)|Action|Drama|Roma...|   108|     2|1042838877|
|    151|Rob Roy (1995)|Action|Drama|Roma...|   109|     4| 841108346|
|    1

Checking the individual counts of `movdf` and `ratedf` dataframes

In [31]:
movdf.count()

9742

In [32]:
ratedf.count()

100836

In [33]:
movdf.select(countDistinct("movieId")).show()

+-----------------------+
|count(DISTINCT movieId)|
+-----------------------+
|                   9742|
+-----------------------+



In [34]:
joindf.filter(joindf.rating == 5).take(10)

[Row(movieId=47, title='Seven (a.k.a. Se7en) (1995)', genres='Mystery|Thriller', userId=1, rating=5, timestamp='964983815'),
 Row(movieId=50, title='Usual Suspects, The (1995)', genres='Crime|Mystery|Thriller', userId=1, rating=5, timestamp='964982931'),
 Row(movieId=101, title='Bottle Rocket (1996)', genres='Adventure|Comedy|Crime|Romance', userId=1, rating=5, timestamp='964980868'),
 Row(movieId=151, title='Rob Roy (1995)', genres='Action|Drama|Romance|War', userId=1, rating=5, timestamp='964984041'),
 Row(movieId=157, title='Canadian Bacon (1995)', genres='Comedy|War', userId=1, rating=5, timestamp='964984100'),
 Row(movieId=163, title='Desperado (1995)', genres='Action|Romance|Western', userId=1, rating=5, timestamp='964983650'),
 Row(movieId=216, title='Billy Madison (1995)', genres='Comedy', userId=1, rating=5, timestamp='964981208'),
 Row(movieId=231, title='Dumb & Dumber (Dumb and Dumber) (1994)', genres='Adventure|Comedy', userId=1, rating=5, timestamp='964981179'),
 Row(movie

#Movie with highest number of 5 star ratings


To do this, first get the movies that are rated 5 stars. Then group them by the `movieId` and get the `count` for each `movieId`

In [35]:
maxStarCountByMovie = joindf.filter(joindf.rating == 5).groupby(col("movieId")).count()

In [36]:
maxStarCountByMovie.take(10)

[Row(movieId=2366, count=5),
 Row(movieId=1580, count=15),
 Row(movieId=1645, count=6),
 Row(movieId=3918, count=2),
 Row(movieId=3175, count=12),
 Row(movieId=32460, count=1),
 Row(movieId=1959, count=1),
 Row(movieId=54190, count=3),
 Row(movieId=471, count=7),
 Row(movieId=6620, count=3)]

Now get the max value of the `count` column through `dataframe.agg()` and `collect()` it which yileds a list.

In [37]:
countlist = maxStarCountByMovie.agg({"count": "max"}).collect()

In [38]:
countlist[0][0]

153

Even if there are multiple entries in the list, since `max` can only be value, extract the max value from the list by index.

In [39]:
maxval = countlist[0][0]

Now crosscheck the `maxStarCountByMovie` dataframe for a row whose count value is `maxval` (153).

In [40]:
maxStarCountByMovie.filter(col("count") == 153).collect()

[Row(movieId=318, count=153)]

Get the title of the movie from `joindf` if you so wish

In [41]:
maxMovId = maxStarCountByMovie.filter(col("count") == 153).collect()

In [42]:
qryId = maxMovId[0][0]

In [43]:
joindf.select("title").where(col("movieId") == qryId).limit(1).collect()

[Row(title='Shawshank Redemption, The (1994)')]

In [44]:
joindf.select("title").where(col("movieId") == qryId).limit(1).collect()[0][0]

'Shawshank Redemption, The (1994)'

#Alternate way to find the movie with maxmimum 5 star rating

In [58]:
topfmov = maxStarCountByMovie.orderBy(col("count"),ascending = False)
topfmov.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    318|  153|
|    296|  123|
|    356|  116|
|   2571|  109|
|    260|  104|
|    527|   92|
|    593|   92|
|    858|   88|
|   2959|   81|
|   1196|   80|
|    110|   80|
|     50|   71|
|   1198|   69|
|   2858|   65|
|   1210|   65|
|    589|   64|
|   4993|   62|
|   2028|   61|
|    608|   57|
|   1197|   55|
+-------+-----+
only showing top 20 rows



From there on, simply get the `movieId` from first row and cross reference it with `joindf` to get the movie title.

#Get the movie with most 0 ratings

Pretty much the same method as above

In [45]:
joindf.filter(col("rating")==0).limit(1).show()

+-------+--------------------+------+------+------+----------+
|movieId|               title|genres|userId|rating| timestamp|
+-------+--------------------+------+------+------+----------+
|     31|Dangerous Minds (...| Drama|     3|     0|1306463578|
+-------+--------------------+------+------+------+----------+



In [46]:
joindf.filter(col("rating") == 0).groupby("movieId").count().limit(10).show()

+-------+-----+
|movieId|count|
+-------+-----+
|   1580|    2|
|   3997|    2|
|   7982|    1|
|    833|    1|
|   1721|    3|
|   1127|    2|
|   4158|    2|
|  56941|    1|
|  74452|    1|
|    737|    1|
+-------+-----+



In [47]:
maxZerodf = joindf.filter(col("rating") == 0).groupby("movieId").count()

In [48]:
maxZerodf.agg({"count": "max"}).collect()

[Row(max(count)=7)]

In [49]:
maxval = maxZerodf.agg({"count": "max"}).collect()[0][0]
maxval

7

In [50]:
maxZerodf.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- count: long (nullable = false)



Get the `movieId` with most Zero ratings

In [51]:
mZeroId = maxZerodf.filter(col("count") == maxval).collect()[0][0]
mZeroId

2683

Get the title of the corresponding `movieId`

In [52]:
joindf.filter(col("movieId") == mZeroId).limit(1).collect()[0][1]

'Austin Powers: The Spy Who Shagged Me (1999)'

#Plot the top 5 movies with 5 star ratings

In [53]:
import matplotlib.pyplot as plt

In [60]:
toplist = topfmov.limit(5).collect()
toplist

[Row(movieId=318, count=153),
 Row(movieId=296, count=123),
 Row(movieId=356, count=116),
 Row(movieId=2571, count=109),
 Row(movieId=260, count=104)]

In [61]:
movlist = []
for x,y in toplist:
  movlist.append(joindf.filter(col("movieId") == x).limit(1).collect()[0][1])

In [62]:
movlist

['Shawshank Redemption, The (1994)',
 'Pulp Fiction (1994)',
 'Forrest Gump (1994)',
 'Matrix, The (1999)',
 'Star Wars: Episode IV - A New Hope (1977)']